## Сорта GLM и как в них разбираться

Импортируем основной набор пакетов, остальные по необходимости позднее:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import statsmodels.formula.api as sf
import numpy as np

### Биномиальная регрессия

* Имя распределения - Бернулли (биномиальное с $n$ = 1)
* Разброс значений ЗП - (0, 1)
* Параметры - $p$ (вероятность успеха), $n$ (количество попыток)
* Типичная функция связи - логит

Прочитаем уже многим знакомые данные с крушения Титаника:

In [ ]:
titanic = pd.read_csv('../../data/titanik_full_data_1.csv', sep = '\t')

In [ ]:
titanic.head()

Из всего этого набора мы возьмём только четыре столбца:

- **Survived** - выжил ли пассажир или нет
- **Sex** - пол пассажира
- **Age** - возраст пассажира
- **Pclass** - класс, в котором плыл пассажир (1, 2 или 3)

Посмотрим на распределение выживших/погибших:

In [ ]:
sns.countplot(x = 'Survived', data = titanic)
plt.xlabel('Выжил ли пассажир')
plt.ylabel('Количество')
plt.title('Судьба пассажиров Титаника')

Применим логистическую регрессию. С помощью `С()` мы указываем категориальные переменные в формуле.

In [ ]:
logit_res = sf.glm('Survived ~ C(Pclass) + C(Sex) + Age', titanic, family = sm.families.Binomial()).fit()

In [ ]:
logit_res.summary()

`Intercept` - шансы выжить (логарифмические) для женщины в первом классе, которой 0 лет. 

- Все коэффициенты значимы (`P>|z|` меньше 0.05)
- Шансы выжить во втором классе ниже, чем в первом, а в третьем ещё ниже
- Быть мужчиной на Титанике ещё хуже
- А также плохо быть старше на Титанике

### Мультиномиальная регрессия

* Имя распределения - мультиномиальное
* Разброс значений - (1...$n$)
* Параметры - $p_1$...$p_n$ (вероятность каждого события), $n$ (количество попыток)
* Типичная функция связи - мультиномиальный логит

In [ ]:
sns.countplot(x = 'Pclass', data = titanic)
plt.xlabel('Класс')
plt.ylabel('Количество')
plt.title('Пассажирские классы')

Строим модель:

In [ ]:
multi_res = sf.mnlogit('Pclass ~ C(Sex) + Age', titanic).fit()
multi_res.summary()

Результат интерпретируем относительно первого класса:

- Мужчин больше во втором, а в третьем ещё больше
- Судя по тому, что интерцепт тоже больше (а в него входят женщины), женщин тоже больше => в других классах просто больше людей
- У возраста обратная зависимость

### Порядковая регрессия

* Имя распределения - кумулятивное пороговое
* Разброс значений - (1...$n$)
* Параметры - $p_1$...$p_n$ (вероятность каждого события)
* Типичная функция связи - порядковый логит

Качаем пакет `bevel` - его нет на `pip`, нужно ставить с [Гитхаба](https://github.com/Shopify/bevel). Про то, как устанавливать пакеты с Гитхаба, читайте [пост на StackOverflow](https://stackoverflow.com/questions/15268953/how-to-install-python-package-from-github).

In [ ]:
from bevel.linear_ordinal_regression import OrderedLogit

В следующем датасете оценивали качество красных вин с разными химическими характеристиками.

In [ ]:
wines = pd.read_csv('D:/Stepik/winequality-red.csv', sep = ';')

In [ ]:
wines.head()

Глянем на распределение рейтингов (переменная `quality`):

In [ ]:
sns.countplot(x = 'quality', data = wines)
plt.xlabel('Оценка')
plt.ylabel('Количество')
plt.title('Рейтинг красных вин')

Так как этот пакет не подерживает формулы, нам нужно выделить ЗП и НП в отдельные переменные:

In [ ]:
Y = wines.quality
X = wines.drop('quality', axis = 1)

Строим модель (здесь сначала идут НП, затем ЗП):

In [ ]:
ol = OrderedLogit()
ol.fit(X, Y)

In [ ]:
ol.print_summary()

`Somers' D` - это как $R^2$, только для порядковых моделей и меняется от -1 до 1 (как корреляция). Чем он больше и положительнее - тем лучше.

Плохо на рейтинги влияют:

- летучая кислотность
- содержание хлоридов
- общее содержание диоксида серы

Хорошо влияет:

- свободный диоксид серы (тут значимость спорная)
- содержание сульфатов
- содержание алкоголя

Ещё есть более богатый на модели и документацию пакет `mord` (документация [вот тут](https://pythonhosted.org/mord/)), который можно скачать через `pip`. Но он настроен на построение предсказательных моделей и не даёт статистического вывода.

### Регрессия количеств

* Имя распределения - Пуассона
* Разброс значений - (0;$\infty$)
* Параметры - $\lambda$ (темп)
* Типичная функция связи - логарифм

Используем данные по владельцам кредитных карточек:

In [ ]:
credit = pd.read_csv('D:/Stepik/credit_card.csv')

In [ ]:
credit.head()

Из этого всего возьмём переменные:

- `active` - количество активных счетов
- `age` - возраст
- `income` - годовой доход в десятках тысяч
- `expenditure` - месячный расход средств кредитной карты
- `owner` - владеет ли пользователь собственным домом или нет
- `selfemp` - самозанятый или нет

Посмотрим распределение количества активных счетов:

In [ ]:
sns.countplot(x = 'active', data = credit)
plt.xlabel('Количество')
plt.ylabel('Частота')
plt.title('Количество активных счетов')

Делаем модель Пуассона:

In [ ]:
pois = sf.glm('active ~ age + income + expenditure + C(owner) + C(selfemp)', \
              family = sm.families.Poisson(), data = credit).fit()
pois.summary()

Количество крединтых кард растёт в зависимости от:

* дохода 
* возраста
* владения собственным домом

#### Проблемы сверхдисперсии

Как подсчитать сверхдисперсию:

In [ ]:
pois.pearson_chi2/pois.df_resid

Для модели Пуассона это значение должно быть близко к 1. Нужно другое распределение, которое может компенсировать проблему:

* Имя распределения - отрицательное биномиальное
* Разброс значений - (0;$\infty$)
* Параметры - $\mu$ (среднее), $\theta$/$\alpha$ (форма/дисперсия)
* Типичная функция связи - логарифм

Когда строим модель, обращаем внимание на аргумент `alpha` - это параметр дисперсии. От него зависит сходимость результатов, поэтому в случае ошибок рекомендуется ставить его в диапазоне от 0.1 до 2.

In [ ]:
neg = sf.glm('active ~ age + income + expenditure + C(owner) + C(selfemp)', data = credit, \
             family = sm.families.NegativeBinomial(alpha=0.15)).fit()
neg.summary()

Интерпретация результатов очень похожа. Что насчёт сверхдисперсии?

In [ ]:
neg.pearson_chi2/neg.df_resid

Сравним модели с помощью информационного критерия Акаике (AIC):

- Его абсолютное значение ничего не значит, полезен только для сравнения моделей
- Чем он ниже, тем лучше модель

Мы можем сравнить два разных типа моделей через AIC (при условии, что ЗП и НП одинаковые):

In [ ]:
print(pois.aic)
print(neg.aic)

Предиктивная способность негативно-биномиальной лучше.

### Регрессия с избытком нулей

* Имя распределения - Пуассона/отрицательное биномиальное с избытком нулей
* Разброс значений - (0;$\infty$)
* Параметры - как у их соответствующих распределений + $\pi$ (вероятность принадлежности нуля одному из двух процессов)
* Типичная функция связи - логарифм

Здесь API не позволяет пользовать формулой, поэтому подготовим данные:

In [ ]:
credit.owner = np.where(credit.owner == 'yes', 1, 0)
credit.selfemp = np.where(credit.selfemp == 'yes', 1, 0) #меняем данные на 0 и 1, чтобы не было ошибки

Y = credit.active #ЗП
X = credit.loc[:, ['owner', 'selfemp', 'age', 'income', 'expenditure']] #НП
X = sm.add_constant(X) # добавляем константу, чтобы в модели был intercept

Чем сложнее модели, тем они капризнее. Увеличиваем количество итераций и меняем алгоритм на более стабильный:

In [ ]:
zeroinf = sm.ZeroInflatedPoisson(Y, X).fit(maxiter = 100, method = 'ncg')
zeroinf.summary()

Резко изменилась интерпретация: значим только возраст и стали значимыми расходы. Ещё появился коэффициент `inflate` - это коэффициент, отвечающий за компенсацию лишних нулей.

Сравним модели:

In [ ]:
print(pois.aic)
print(neg.aic)
print(zeroinf.aic)

И то же самое с отрицательным биномиальным:

In [ ]:
zeroinf_2 = sm.ZeroInflatedNegativeBinomialP(Y, X).fit(maxiter = 100, method = 'ncg')
zeroinf_2.summary()

А здесь интерпретация похожа на изначальную. Ещё есть параметр `alpha`, который оценивает избыток дисперсии.

Сравним:

In [ ]:
print(pois.aic)
print(neg.aic)
print(zeroinf.aic)
print(zeroinf_2.aic)

Zero-inflated лучше своих обычных вариантов в данной ситуации.

### Анализ выживаемости

* Имя распределения - Вейбулла (как пример)
* Разброс значений - (0;$\infty$)
* Параметры - $\alpha$ (дисперсия), $\gamma$ (форма)
* Типичная функция связи - логарифм

Для этого мы используем пакет [lifelines](https://lifelines.readthedocs.io/en/latest/), устанавливается через `pip`.

In [ ]:
import lifelines as lf

Будем испытывать его на данных оттока клиентов:

In [ ]:
churn = pd.read_csv('https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv')

In [ ]:
churn.head()

Возьмём несколько из них:

In [ ]:
churn = churn.loc[:, ['Churn', 'tenure', 'SeniorCitizen', 'Dependents', 'MonthlyCharges', 'PaperlessBilling']]

In [ ]:
churn.head()

- **Churn** - ушёл клиент или нет
- **tenure** - сколько месяцев пробыл с компанией
- **SeniorCitizen** - клиент пожилой или нет
- **Dependents** - есть иждивенцы в семье или нет
- **MonthlyCharges** - сколько клиент платит в месяц
- **PaperlessBilling** - оплата с чеком или бесчековая

Распределение оттока:

In [ ]:
sns.countplot(x = 'Churn', data = churn)
plt.xlabel('Ушёл ли клиент?')
plt.ylabel('Количество')
plt.title('Судьба клиентов компании')

Распределение, сколько люди месяцев проводят с компанией:

In [ ]:
sns.distplot(churn.tenure, kde = False)
plt.xlabel('Количество месяцев')
plt.ylabel('Частота')
plt.title('Сколько времени клиенты провели с компанией')

Предварительно подготовим данные:

In [ ]:
churn.tenure = churn.tenure + 0.001 #чтобы не было нулевых месяцев
churn.Churn = np.where(churn.Churn == 'Yes', 1, 0) #перекодируем в числа
churn.SeniorCitizen = np.where(churn.SeniorCitizen == 1, 'Yes', 'No') #наоборот

Строим модель. Ей нужно указать данные, колонку со временем, колонку с событием, и опционально формулу.

In [ ]:
surv = lf.WeibullAFTFitter()

surv.fit(df = churn, duration_col = 'tenure', event_col = 'Churn', \
         formula = 'C(SeniorCitizen) + C(Dependents) + MonthlyCharges + C(PaperlessBilling)')

In [ ]:
surv.print_summary()

Если коэффициент меньше нуля - время до события короче.
Если коэффициент больше нуля - время до события дольше.

Интерпретация:

* Пожилые люди меньше времени пользуются сервисом
* Те, у кого бесчековая оплата - ещё меньше

* Люди с иждивенциами пользуются им больше обычного

Для интересующихся, что такое **-log2(p)**, читать вот [эту статью](https://lesslikely.com/statistics/s-values).

### TL;DR

* Переменная из двух категорий – биномиальная регрессия
* Категорий больше – мультиномиальная
* В категориях есть явное убывание или нарастание – порядковая
* Считаем количество чего-то – Пуассоновская
  - Дисперсия больше среднего – отрицательно-биномиальная
  Слишком много нулей – zero-inflated-модель
* У нас есть какое-то событие и время до него – анализ выживаемости
